# Zoom-in sampling of coro including a phase ramp for shifts

In [ ]:
import os
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from Asterix.optics import butterworth_circle, fqpm_mask, mft, prop_fpm_regional_sampling, fft_choosecenter, create_wrapped_vortex_mask
from Asterix.optics import phase_amplitude_functions as paf
from Asterix.utils.plotting import display_complex

In [ ]:
def shift_phase_ramp(dim_pp, shift_x, shift_y):
    """
    Create a phase ramp of size (dim_pp,dim_pp) that can be used as follow
    to shift one image by (a,b) pixels : shift_im = real(fft(ifft(im)*exp(i phase ramp)))

    AUTHOR: Axel Potier

    Parameters
    ----------
    dim_pp : int
        Size of the phase ramp (in pixels)
    shift_x : float
        Shift desired in the x direction (in pixels)
    shift_y : float
        Shift desired in the y direction (in pixels)

    Returns
    ------
    masktot : 2D array
        Phase ramp
    """
    if (shift_x == 0) & (shift_y == 0):
        ramp = 1
    else:
        maskx = np.linspace(-np.pi * shift_x, np.pi * shift_x, dim_pp, endpoint=False)
        masky = np.linspace(-np.pi * shift_y, np.pi * shift_y, dim_pp, endpoint=False)
        xx, yy = np.meshgrid(maskx, masky)
        ramp = np.exp(-1j * xx) * np.exp(-1j * yy)
    return ramp

## Set up inputs

In [ ]:
dim = 400
ray = dim / 2
shift = (0, 0)  # lambda/D

nbrs_list = [0.1, 2., 10., 100.]
nbrs_list2 = [0.1, 5., 30., 200.]
nbrs_list3 = [40., 80., 120., 160.]

In [ ]:
# Create pupil aperture and Lyot stop
pup = paf.roundpupil(dim, ray, grey_pup_bin_factor=10)
lyot_stop = paf.roundpupil(dim, ray*0.95, grey_pup_bin_factor=1)

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(pup, cmap='Greys_r')
plt.title('Grey pupil')
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(lyot_stop, cmap='Greys_r')
plt.title('Lyot stop')
plt.colorbar()

In [ ]:
# Choose FPM
fpm = fqpm_mask(pup.shape[0])
# thval = np.array([0, 3, 4, 5, 8]) * np.pi / 8
# phval = np.array([3, 0, 1, 2, 1]) * np.pi
# jump = np.array([2, 2, 2, 2]) * np.pi
# _, fpm = create_wrapped_vortex_mask(dim=pup.shape[0],
#                                      thval=thval,
#                                      phval=phval,
#                                      jump=jump)

plt.imshow(fpm, cmap='Reds')
plt.title('Phase')
plt.colorbar()

## Test built-in function

In [ ]:
psam_pre_ls_builtin = prop_fpm_regional_sampling(pup, np.exp(1j*fpm), nbrs_list, shift=shift)

display_complex(psam_pre_ls_builtin)
plt.suptitle('Pre-LS E-field')

In [ ]:
plt.imshow(np.log10(np.abs(psam_pre_ls_builtin)**2), cmap='inferno')#, norm=LogNorm())

Just look at its MFT

In [ ]:
this = mft(psam_pre_ls_builtin, 400, 400, 64)
plt.imshow(np.abs(this)**2, cmap='inferno', norm=LogNorm())

## Break up function

In [ ]:
dim = pup.shape[0]

fpm_z = np.exp(1.j * fpm)
nbres = res_list
samp_outer = 2
filter_order = 15
alpha = 1.5

print(dim)
print(nbres)

### Pre-compute phase ramp

In [ ]:
phase_ramp = shift_phase_ramp(dim, shift[0], shift[1])
inverted_phase_ramp = shift_phase_ramp(dim, -px_shift_x, -px_shift_y)

pup0 = np.array(pup, copy=True, dtype='complex128')

### Inner part of FPM

In [ ]:
# Butterworth filter
but0 = butterworth_circle(dim, dim / alpha, filter_order, -0.5, -0.5)
display_complex(but0)

In [ ]:
# E-field before the FPM in inner part of focal plane
efield_before_fpm = mft(pup0 * phase_ramp, real_dim_input=dim, dim_output=dim, nbres=nbres[0])
display_complex(efield_before_fpm)

print(f"nbres={nbres[0]}")
print(np.where(np.abs(efield_before_fpm) == np.max(np.abs(efield_before_fpm))))

In [ ]:
plt.imshow(np.angle(phase_ramp), cmap='RdBu')
plt.colorbar()

In [ ]:
# Total E-field before the LS
efield_before_ls = mft(efield_before_fpm * fpm_z * but0, real_dim_input=dim, dim_output=dim, nbres=nbres[0],
                       inverse=True) * inverted_phase_ramp
display_complex(efield_before_ls)

### Layers of FPM sampling

In [ ]:
but_list = []
pre_fpm_list = []
pre_ls_list = []

In [ ]:
const_but = butterworth_circle(dim, dim / alpha, filter_order, xshift=-0.5, yshift=-0.5)

for k in range(nbres.shape[0] - 1):
    print(f"for k={k}, nbres={nbres[k + 1]}")

    # Butterworth filter in each layer
    sizebut_here = dim / alpha * nbres[k] / nbres[k + 1]
    but = (1 - butterworth_circle(dim, sizebut_here, filter_order, xshift=-0.5, yshift=-0.5)) * const_but
    but_list.append(but)
    
    # E-field before the FPM in each layer    
    ef_pre_fpm = mft(pup0 * phase_ramp, real_dim_input=dim, dim_output=dim, nbres=nbres[k + 1])
    pre_fpm_list.append(ef_pre_fpm)

    # E-field before the LS in each layer
    ef_pre_ls = mft(ef_pre_fpm * fpm_z * but, real_dim_input=dim, dim_output=dim, nbres=nbres[k + 1],
                    inverse=True) * inverted_phase_ramp
    pre_ls_list.append(ef_pre_ls)

In [ ]:
# Plot all filters
plt.figure(figsize=(15,7))
for k in range(3):
    plt.subplot(2,3, k+1)
    plt.imshow(np.abs(but_list[k])**2, cmap='Greys_r')
    plt.colorbar()
    plt.title(f"Intensity, k={k}")
    plt.subplot(2,3, k+4)
    plt.imshow(np.angle(but_list[k]), cmap='RdBu')
    plt.colorbar()
    plt.title(f"Phase, k={k}")

In [ ]:
# MFT of butterworth filter
print(dim / samp_outer)
that0 = mft(but_list[0], 400, 400, 256)
that1 = mft(but_list[1], 400, 400, 256)
that2 = mft(but_list[2], 400, 400, 256)

In [ ]:
plt.imshow(np.abs(that0)**2, cmap='inferno', norm=LogNorm())

The filters are supposed to be the same. But the sampling will change when applied, so that the E-field won't be the same. The ratio between the elements of `nbres` used is the same in our case.

In [ ]:
# Plot all E-fields before the FPM
plt.figure(figsize=(15,7))
for k in range(3):
    plt.subplot(2,3,k+1)
    plt.imshow(np.abs(pre_fpm_list[k])**2, cmap='inferno', norm=LogNorm())
    plt.colorbar()
    plt.title(f"Intensity, k={k}, nbres={nbres[k+1]}")
    plt.subplot(2,3,k+4)
    plt.imshow(np.angle(pre_fpm_list[k]), cmap='RdBu')
    plt.colorbar()
    plt.title(f"Phase, k={k}")

In [ ]:
# Analyze realized shifts of the PSF in each of the intensity images
# to check the correct phase ramp has been used in all layers

cen = pre_fpm_list[0].shape[0] / 2
print(f"cen = {cen}\n")

plt.figure(figsize=(15, 5))
for k in range(3):
    plt.subplot(1, 3, k+1)
    im = np.abs(pre_fpm_list[k])**2
    plt.imshow(im, cmap='inferno', norm=LogNorm())
    plt.colorbar()
    plt.title(f"Intensity, k={k}, nbres={nbres[k+1]}")
    
    print(f"nbres={nbres[k+1]}")
    print(f"Measured px shift: {cen - np.where(im == np.max(im))[1]}")
    print('\n')

In [ ]:
# Plot filter over pre-FPM PSF
n = 0
plt.imshow(np.abs(but_list[n] * pre_fpm_list[n])**2, cmap='inferno', norm=LogNorm())
plt.colorbar()

In [ ]:
# Plot all E-fields before the LS (intensity and phase)
plt.figure(figsize=(15,7))
for k in range(3):
    plt.subplot(2,3,k+1)
    plt.imshow(np.abs(pre_ls_list[k])**2, cmap='inferno', norm=LogNorm())
    plt.colorbar()
    plt.title(f"Intensity, k={k}")
    plt.subplot(2,3,k+4)
    plt.imshow(np.angle(pre_ls_list[k]), cmap='RdBu')
    plt.colorbar()
    plt.title(f"Phase, k={k}")

In [ ]:
# Plot all E-fields before the LS (absolute and real)
plt.figure(figsize=(15,7))
for k in range(3):
    plt.subplot(2,3,k+1)
    plt.imshow(np.abs(pre_ls_list[k]), cmap='inferno')
    plt.colorbar()
    plt.title(f"Absolute value, k={k}")
    plt.subplot(2,3,k+4)
    plt.imshow(np.real(pre_ls_list[k]), cmap='inferno')
    plt.colorbar()
    plt.title(f"Real part, k={k}")

In [ ]:
# Total E-field before the LS
summed_layers_before_ls = efield_before_ls + np.sum(np.array(pre_ls_list), axis=0)
display_complex(summed_layers_before_ls)
plt.suptitle("Summed pre-LS E-field, through all layers (except outermost)")

In [ ]:
plt.imshow(np.abs(summed_layers_before_ls)**2, cmap='inferno', norm=LogNorm())
plt.colorbar()
plt.title("Summed pre-LS E-field, through all layers (except outermost)")

In [ ]:
plt.figure(figsize=(16,8))

plt.subplot(2,6,1)
plt.imshow(np.log10(np.abs(efield_before_ls)**2), cmap='inferno')#, norm=LogNorm())
plt.title('efield_before_ls')

plt.subplot(2,6,2)
plt.imshow(np.abs(pre_ls_list[0])**2, cmap='inferno', norm=LogNorm())
plt.title('pre_ls_list[0]')

plt.subplot(2,6,3)
plt.imshow(np.abs(pre_ls_list[1])**2, cmap='inferno', norm=LogNorm())
plt.title('pre_ls_list[1]')

plt.subplot(2,6,4)
plt.imshow(np.abs(pre_ls_list[2])**2, cmap='inferno', norm=LogNorm())
plt.title('pre_ls_list[2]')

plt.subplot(2,6,5)
plt.imshow(np.log10(np.abs(np.sum(np.array(pre_ls_list), axis=0))**2), cmap='inferno')#, norm=LogNorm())
plt.title('np.sum(np.array(pre_ls_list), axis=0)')

plt.subplot(2,6,6)
plt.imshow(np.log10(np.abs(summed_layers_before_ls)**2), cmap='inferno')#, norm=LogNorm())
plt.title('summed_layers_before_ls')

plt.subplot(2,6,7)
plt.imshow(np.angle(efield_before_ls), cmap='RdBu', norm=LogNorm())
plt.title('efield_before_ls')

plt.subplot(2,6,8)
plt.imshow(np.angle(pre_ls_list[0]), cmap='RdBu', norm=LogNorm())
plt.title('pre_ls_list[0]')

plt.subplot(2,6,9)
plt.imshow(np.angle(pre_ls_list[1]), cmap='RdBu', norm=LogNorm())
plt.title('pre_ls_list[1]')

plt.subplot(2,6,10)
plt.imshow(np.angle(pre_ls_list[2]), cmap='RdBu', norm=LogNorm())
plt.title('pre_ls_list[2]')

plt.subplot(2,6,11)
plt.imshow(np.angle(np.sum(np.array(pre_ls_list), axis=0)), cmap='RdBu', norm=LogNorm())
plt.title('np.sum(np.array(pre_ls_list), axis=0)')

plt.subplot(2,6,12)
plt.imshow(np.angle(summed_layers_before_ls), cmap='RdBu', norm=LogNorm())
plt.title('summed_layers_before_ls')

### Outer part of FPM

In [ ]:
# Butterworth filter in outer part of focal plane
nbres_outer = dim / samp_outer
sizebut_outer = dim / alpha * nbres[-1] / nbres_outer
but_outer = 1 - butterworth_circle(dim, sizebut_outer, filter_order, xshift=-0.5, yshift=-0.5)
display_complex(but_outer)

print(nbres[-1])
print(samp_outer)
print(nbres_outer)

In [ ]:
# E-field before the FPM in outer part of focal plane    
ef_pre_fpm_outer = mft(pup0 * phase_ramp, real_dim_input=dim, dim_output=dim, nbres=nbres_outer,
                       inverse=False)
display_complex(ef_pre_fpm_outer)

In [ ]:
# E-field before the LS in outer part of focal plane
ef_pre_ls_outer = mft(ef_pre_fpm_outer * fpm_z * but_outer, real_dim_input=dim, dim_output=dim, nbres=nbres_outer,
                      inverse=True) * inverted_phase_ramp
display_complex(ef_pre_ls_outer)

In [ ]:
np.max(np.abs(ef_pre_ls_outer)) / np.max(np.abs(summed_layers_before_ls))

In [ ]:
plt.imshow(np.abs(ef_pre_ls_outer)**2, cmap='inferno', norm=LogNorm())
plt.colorbar()
plt.title("Summed pre-LS E-field, only outermost layer")

In [ ]:
# Total E-field before the LS
psam_pre_ls_nb = summed_layers_before_ls + ef_pre_ls_outer
display_complex(psam_pre_ls_nb)

In [ ]:
plt.imshow(np.log10(np.abs(psam_pre_ls_nb)**2), cmap='inferno')#, norm=LogNorm())

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(np.log10(np.abs(psam_pre_ls_nb)))
plt.title('Notebook')
plt.subplot(1,2,2)
plt.imshow(np.log10(np.abs(psam_pre_ls_builtin)))
plt.title('Built-in function')

print(np.sum(psam_pre_ls_nb))
print(np.sum(psam_pre_ls_builtin))

## Calc PSFs

In [ ]:
#post_ls = psam_pre_ls_nb * lyot_stop
post_ls = psam_pre_ls_builtin * lyot_stop

display_complex(post_ls)
plt.suptitle('E-field after LS')

In [ ]:
lamD_psf = 4

In [ ]:
direct_ef = mft(pup*lyot_stop, real_dim_input=dim, dim_output=dim, nbres=dim/lamD_psf)
direct_psf = np.abs(direct_ef)**2
norm = direct_psf.max()
print(f"norm = {norm}")

plt.imshow(direct_psf / norm, cmap='inferno', norm=LogNorm())
plt.title('Direct PSF')
plt.colorbar()

In [ ]:
#coro_ef = mft(post_ls, real_dim_input=dim, dim_output=dim, nbres=dim/lamD_psf)
coro_ef = mft(post_ls, real_dim_input=400, dim_output=400, nbres=64)
coro_psf = np.abs(coro_ef)**2

plt.imshow(coro_psf / norm, cmap='inferno', norm=LogNorm())
plt.title('Coronagraphic PSF')
plt.colorbar()

In [ ]:
plt.imshow(coro_psf[100:410, 100:410] / norm, cmap='inferno', norm=LogNorm())
#plt.imshow(coro_psf[200:310, 200:310] / norm, cmap='inferno', norm=LogNorm())
plt.title('Coronagraphic PSF')
plt.colorbar()

In [ ]:
np.max(coro_psf / norm)

## Improving on the residual energy - testing

In [ ]:
dim = 400
rad = dim / 2
samp_outer = 2

pup = paf.roundpupil(dim, rad, grey_pup_bin_factor=10)
lyot_stop = paf.roundpupil(dim, rad * 0.95)
fpm = fqpm_mask(dim)

res_list = np.array([0.1, 1, 10, 100])
pre_ls_areas = prop_fpm_regional_sampling(pup, np.exp(1j * fpm), nbrs_list)
post_ls_areas = pre_ls_areas * lyot_stop

nbres_direct = dim / samp_outer
pre_fpm = mft(pup, real_dim_input=dim, dim_output=dim, nbres=nbres_direct)
post_fpm = pre_fpm * np.exp(1j * fpm)
pre_ls_direct = mft(post_fpm, real_dim_input=dim, dim_output=dim, nbres=nbres_direct, inverse=True)
post_ls_direct = pre_ls_direct * lyot_stop

In [ ]:
direct_ef = mft(pup*lyot_stop, real_dim_input=dim, dim_output=dim, nbres=nbres_direct)
direct_psf = np.abs(direct_ef)**2
norm = direct_psf.max()

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(np.abs(post_ls_areas), cmap='inferno', norm=LogNorm())
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(np.abs(post_ls_direct), cmap='inferno', norm=LogNorm())
plt.colorbar()

In [ ]:
print(np.sum(np.abs(post_ls_areas)**2))
print(np.sum(np.abs(post_ls_direct)**2))

In [ ]:
coro_ef_areas = mft(post_ls_areas, real_dim_input=dim, dim_output=dim, nbres=nbres_direct)
coro_psf_areas = np.abs(coro_ef_areas) ** 2 / norm

In [ ]:
coro_ef_direct = mft(post_ls_direct, real_dim_input=dim, dim_output=dim, nbres=nbres_direct)
coro_psf_direct = np.abs(coro_ef_direct) ** 2 / norm

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(coro_psf_areas, cmap='inferno', norm=LogNorm())
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(coro_psf_direct, cmap='inferno', norm=LogNorm())
plt.colorbar()

In [ ]:
assert (np.max(coro_psf_direct) / np.max(coro_psf_areas)) > 1e4